1) Mount Colab.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


2) Inspect the thoroughly cleaned train dataset.

In [ ]:
import pandas as pd

# Load dataset
train_dir = "/content/drive/MyDrive/Big Data Analysis and Project - a1906525/Datasets/crime_train_final.csv"
df = pd.read_csv(train_dir)

# Check datatypes
print(df.dtypes)

# Count numerical and categorical columns
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object', 'category', 'bool']).columns

print(f"Numerical columns ({len(num_cols)}): {list(num_cols)}")
print(f"Categorical columns ({len(cat_cols)}): {list(cat_cols)}")

highest_offense_description     object
highest_offense_code             int64
family_violence                 object
occurred_date_time_year          int64
sine_odt_month                 float64
cosine_odt_month               float64
sine_odt_week_of_year          float64
cosine_odt_week_of_year        float64
sine_odt_day                   float64
cosine_odt_day                 float64
sine_odt_day_of_week           float64
cosine_odt_day_of_week         float64
sine_odt_hour                  float64
cosine_odt_hour                float64
sine_odt_minute                float64
cosine_odt_minute              float64
sine_occurred_time             float64
cosine_occurred_time           float64
sine_report_time               float64
cosine_report_time             float64
report_date_time_year            int64
sine_rdt_week_of_year          float64
cosine_rdt_week_of_year        float64
sine_rdt_day                   float64
cosine_rdt_day                 float64
sine_rdt_hour            

3) Convert the apd_district and highest_offense_code alone to categorical.

In [ ]:
cat_convert = ['apd_district', 'highest_offense_code']

for col in cat_convert:
    df[col] = df[col].astype('category')

print(df[cat_convert].dtypes)

apd_district            category
highest_offense_code    category
dtype: object


4) Load and inspect the test dataset.

In [ ]:
import pandas as pd

# Load dataset
test_dir = "/content/drive/MyDrive/Big Data Analysis and Project - a1906525/Datasets/crime_test_cleaned.csv"
dft = pd.read_csv(test_dir)

# Check datatypes
print(dft.dtypes)

# Count numerical and categorical columns
num_cols = dft.select_dtypes(include=['int64', 'float64']).columns
cat_cols = dft.select_dtypes(include=['object', 'category', 'bool']).columns

print(f"Numerical columns ({len(num_cols)}): {list(num_cols)}")
print(f"Categorical columns ({len(cat_cols)}): {list(cat_cols)}")

highest_offense_description     object
highest_offense_code             int64
family_violence                 object
occurred_date_time_year          int64
sine_odt_month                 float64
cosine_odt_month               float64
sine_odt_week_of_year          float64
cosine_odt_week_of_year        float64
sine_odt_day                   float64
cosine_odt_day                 float64
sine_odt_day_of_week           float64
cosine_odt_day_of_week         float64
sine_odt_hour                  float64
cosine_odt_hour                float64
sine_odt_minute                float64
cosine_odt_minute              float64
sine_occurred_time             float64
cosine_occurred_time           float64
sine_report_time               float64
cosine_report_time             float64
report_date_time_year            int64
sine_rdt_week_of_year          float64
cosine_rdt_week_of_year        float64
sine_rdt_day                   float64
cosine_rdt_day                 float64
sine_rdt_hour            

5) Convert those columns back to categorical as we face the same problem with the test dataset as well.

In [ ]:
cat_convert = ['apd_district', 'highest_offense_code']

for col in cat_convert:
    dft[col] = dft[col].astype('category')

print(dft[cat_convert].dtypes)

apd_district            category
highest_offense_code    category
dtype: object


6) Check for NAs in both of them.

In [ ]:
import pandas as pd

# Load train
train_dir = "/content/drive/MyDrive/Big Data Analysis and Project - a1906525/Datasets/crime_train_final.csv"
df_train = pd.read_csv(train_dir)

# Load test
test_dir = "/content/drive/MyDrive/Big Data Analysis and Project - a1906525/Datasets/crime_test_cleaned.csv"
df_test = pd.read_csv(test_dir)

# Check for NAs in train
if df_train.isna().sum().sum() == 0:
    print("No NAs in train dataset.")
else:
    print("There are missing values in train dataset.")

# Check for NAs in test
if df_test.isna().sum().sum() == 0:
    print("No NAs in test dataset.")
else:
    print("There are missing values in test dataset.")

No NAs in train dataset.
No NAs in test dataset.


7) Java Script for keeping code alive.

In [ ]:
from IPython.display import Javascript, display

# Inject JS that clicks the "Connect"/"Reconnect" button every minute
js_code = """
function ClickConnect() {
  const btn = document.querySelector('colab-connect-button') ||
              document.querySelector('#top-toolbar > colab-connect-button');
  if (btn) {
    // Click the inner shadow-root's "connect" button if needed
    const inner = btn.shadowRoot ? btn.shadowRoot.querySelector('#connect') : null;
    (inner || btn).click();
    console.log('🟢 ClickConnect clicked at', new Date().toLocaleTimeString());
  } else {
    console.log('⚠️ Connect button not found');
  }
}
setInterval(ClickConnect, 60000);
"""
display(Javascript(js_code))

8) Modelling - Tab Transformer

Note: The balanced accuracy, and recall are lame parameters. So they were not used in the report.

In [ ]:
!pip install tab-transformer-pytorch==0.4.2

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

import torch, random, numpy as np, pandas as pd
from torch.amp import autocast, GradScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, balanced_accuracy_score, confusion_matrix)
import matplotlib.pyplot as plt
import seaborn as sns
from tab_transformer_pytorch import TabTransformer

# Reproducibility
SEED = 1906525
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True; torch.backends.cudnn.benchmark = False

# Paths
train_path = "/content/drive/MyDrive/Big Data Analysis and Project - a1906525/Datasets/crime_train_final.csv"
test_path = "/content/drive/MyDrive/Big Data Analysis and Project - a1906525/Datasets/crime_test_cleaned.csv"
perf_dir = "/content/drive/MyDrive/Big Data Analysis and Project - a1906525/Results/Tab Transformer/Performance Metrics"
cm_dir = "/content/drive/MyDrive/Big Data Analysis and Project - a1906525/Results/Tab Transformer/Confusion Matrices"
test_pred_dir = "/content/drive/MyDrive/Big Data Analysis and Project - a1906525/Results/Tab Transformer/Test Results"
for d in [perf_dir, cm_dir, test_pred_dir]:
    os.makedirs(d, exist_ok=True)

# Data load & preprocessing
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
cat_cols = ['highest_offense_description','family_violence','location_type',
            'apd_sector','clearance_status','apd_district','highest_offense_code']
for c in cat_cols:
    df_train[c] = df_train[c].astype('category')
    df_test[c] = df_test[c].astype('category')
features = [c for c in df_train.columns if c != "clearance_status"]
target = "clearance_status"
feature_cat_cols = [c for c in cat_cols if c != target]
for c in feature_cat_cols:
    df_train[c] = df_train[c].cat.codes
    df_test[c]  = df_test[c].cat.codes

cat_dims = [df_train[c].nunique() for c in feature_cat_cols]
num_cont = len([c for c in features if c not in feature_cat_cols])
labels = list(df_train[target].cat.categories)

train, val = train_test_split(df_train, test_size=0.2,
                               stratify=df_train[target], random_state=SEED)

# Arrays
X_train_cat = train[feature_cat_cols].values.astype(np.int64)
X_val_cat   = val[feature_cat_cols].values.astype(np.int64)
X_test_cat  = df_test[feature_cat_cols].values.astype(np.int64)
X_train_num = train[[c for c in features if c not in feature_cat_cols]].values.astype(np.float32)
X_val_num   = val[[c for c in features if c not in feature_cat_cols]].values.astype(np.float32)
X_test_num  = df_test[[c for c in features if c not in feature_cat_cols]].values.astype(np.float32)
y_train = train[target].cat.codes.values.astype(np.int64)
y_val   = val[target].cat.codes.values.astype(np.int64)
y_test  = df_test[target].cat.codes.values.astype(np.int64)

def compute_metrics(y_true, y_pred, labels):
    cm = confusion_matrix(y_true, y_pred, labels=range(len(labels)))
    if cm.shape == (2,2):
        tn, fp, fn, tp = cm.ravel()
        spec = tn/(tn+fp) if (tn+fp) else 0
        sens = tp/(tp+fn) if (tp+fn) else 0
    else:
        spec = sens = 0
    return cm, spec, sens

def get_metrics(y_true, y_pred, probs, labels):
    acc = accuracy_score(y_true, y_pred)
    loss = -np.mean(np.log(probs[np.arange(len(y_true)), y_true] + 1e-12))
    prec = precision_score(y_true, y_pred, pos_label=labels.index("C"), average='binary')
    rec  = recall_score(y_true, y_pred, pos_label=labels.index("C"), average='binary')
    f1   = f1_score(y_true, y_pred, pos_label=labels.index("C"), average='binary')
    bal  = balanced_accuracy_score(y_true, y_pred)
    cm, spec, sens = compute_metrics(y_true, y_pred, labels)
    return acc, loss, prec, rec, f1, bal, spec, sens, cm

# Training config
embed_list = [8,16,32,64,128]
epochs_list = [5,10,20,35,50]
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 512
accum_steps = 4
all_results = []

for ed in embed_list:
    for ep in epochs_list:
        print(f"\nRunning: embed={ed}, epochs={ep}")
        model = TabTransformer(
            categories=tuple(cat_dims),
            num_continuous=num_cont,
            dim=ed, dim_out=len(labels),
            depth=2, heads=4,
            attn_dropout=0.1, ff_dropout=0.1
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        criterion = torch.nn.CrossEntropyLoss()
        scaler = GradScaler()

        model.train()
        N = len(y_train)
        idxs = np.arange(N)
        for epoch in range(1, ep+1):
            np.random.shuffle(idxs)
            optimizer.zero_grad()
            for i in range(0, N, batch_size):
                batch = idxs[i:i+batch_size]
                xb_c = torch.tensor(X_train_cat[batch]).to(device)
                xb_n = torch.tensor(X_train_num[batch]).to(device)
                yb = torch.tensor(y_train[batch]).to(device)
                with autocast(device_type="cuda"):
                    logits = model(xb_c, xb_n)
                    loss = criterion(logits, yb)/accum_steps
                scaler.scale(loss).backward()
                if (i//batch_size +1) % accum_steps==0 or i+batch_size>=N:
                    scaler.step(optimizer); scaler.update(); optimizer.zero_grad()
            # ------- Insert: Accuracy print for this epoch -------
            model.eval()
            with torch.no_grad(), autocast(device_type="cuda"):
                # Train acc
                logits_train = model(torch.tensor(X_train_cat).to(device), torch.tensor(X_train_num).to(device))
                preds_train = logits_train.argmax(dim=1).cpu().numpy()
                train_acc = accuracy_score(y_train, preds_train)
                # Test acc
                logits_test = model(torch.tensor(X_test_cat).to(device), torch.tensor(X_test_num).to(device))
                preds_test = logits_test.argmax(dim=1).cpu().numpy()
                test_acc = accuracy_score(y_test, preds_test)
            print(f"Epoch {epoch}/{ep} | embed={ed}: Train Acc={train_acc:.4f} | Test Acc={test_acc:.4f}")
            model.train()
            # -----------------------------------------------------
        print(f"\nEmbed={ed}, epochs={ep} done")

        model.eval()
        with torch.no_grad(), autocast(device_type="cuda"):
            logits_train = model(torch.tensor(X_train_cat).to(device), torch.tensor(X_train_num).to(device))
            logits_val   = model(torch.tensor(X_val_cat).to(device), torch.tensor(X_val_num).to(device))
            logits_test  = model(torch.tensor(X_test_cat).to(device), torch.tensor(X_test_num).to(device))

        preds_train = logits_train.argmax(dim=1).cpu().numpy()
        preds_val   = logits_val.argmax(dim=1).cpu().numpy()
        preds_test  = logits_test.argmax(dim=1).cpu().numpy()
        probs_train = torch.softmax(logits_train, dim=1).cpu().numpy()
        probs_val   = torch.softmax(logits_val, dim=1).cpu().numpy()
        probs_test  = torch.softmax(logits_test, dim=1).cpu().numpy()

        ta, tl, _, _, _, _, _, _, _ = get_metrics(y_train, preds_train, probs_train, labels)
        va, vl, _, _, _, _, _, _, _ = get_metrics(y_val, preds_val, probs_val, labels)
        test_acc, test_loss, prec, rec, f1, bal_acc, spec, sens, cm = get_metrics(y_test, preds_test, probs_test, labels)

        all_results.append({
            'Model': 'TabTransformer', 'embedding_dim': ed, 'epochs': ep,
            'train_accuracy': ta, 'val_accuracy': va, 'test_accuracy': test_acc,
            'train_loss': tl, 'val_loss': vl, 'test_loss': test_loss,
            'precision': prec, 'recall': rec, 'f1': f1,
            'balanced_accuracy': bal_acc, 'specificity': spec, 'sensitivity': sens
        })

        fname = f"ed{ed}_ep{ep}"
        cm_path = os.path.join(cm_dir, fname + ".png")
        pred_path = os.path.join(test_pred_dir, fname + "_predicted.csv")

        plt.figure(figsize=(4,4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=labels, yticklabels=labels)
        plt.xlabel('Predicted'); plt.ylabel('Actual')
        plt.title(f"CM – embed={ed}, ep={ep}")
        plt.tight_layout(); plt.savefig(cm_path); plt.close()

        sample = df_test.copy()
        sample['predicted_clearance_status'] = [labels[p] for p in preds_test]
        cols = list(sample.columns)
        cols.insert(cols.index('clearance_status')+1, cols.pop(cols.index('predicted_clearance_status')))
        sample.sample(15).to_csv(pred_path, index=False)

        print(f"Saved CM at {cm_path}, preds at {pred_path}, test_acc={test_acc:.4f}")

        del model, optimizer, scaler
        torch.cuda.empty_cache()


# Save summary
df_all = pd.DataFrame(all_results)
summary_path = os.path.join(perf_dir, "TabTransformer_all_results.xlsx")
df_all.to_excel(summary_path, index=False)
print(f"\nAll hyperparameter runs complete. Summary at {summary_path}")


Running: embed=8, epochs=5
Epoch 1/5 | embed=8: Train Acc=0.8438 | Test Acc=0.6310
Epoch 2/5 | embed=8: Train Acc=0.8579 | Test Acc=0.5910
Epoch 3/5 | embed=8: Train Acc=0.8647 | Test Acc=0.5394
Epoch 4/5 | embed=8: Train Acc=0.8682 | Test Acc=0.5679
Epoch 5/5 | embed=8: Train Acc=0.8691 | Test Acc=0.6115

Embed=8, epochs=5 done
Saved CM at /content/drive/MyDrive/Big Data Analysis and Project - a1906525/Results/Tab Transformer/Confusion Matrices/ed8_ep5.png, preds at /content/drive/MyDrive/Big Data Analysis and Project - a1906525/Results/Tab Transformer/Test Results/ed8_ep5_predicted.csv, test_acc=0.6115

Running: embed=8, epochs=10
Epoch 1/10 | embed=8: Train Acc=0.8477 | Test Acc=0.7030
Epoch 2/10 | embed=8: Train Acc=0.8626 | Test Acc=0.6688
Epoch 3/10 | embed=8: Train Acc=0.8657 | Test Acc=0.6826
Epoch 4/10 | embed=8: Train Acc=0.8675 | Test Acc=0.6817
Epoch 5/10 | embed=8: Train Acc=0.8699 | Test Acc=0.6514
Epoch 6/10 | embed=8: Train Acc=0.8702 | Test Acc=0.7057
Epoch 7/10 | emb

/tmp/ipython-input-20-1905850978.py:73: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(np.log(probs[np.arange(len(y_true)), y_true] + 1e-12))


Saved CM at /content/drive/MyDrive/Big Data Analysis and Project - a1906525/Results/Tab Transformer/Confusion Matrices/ed16_ep35.png, preds at /content/drive/MyDrive/Big Data Analysis and Project - a1906525/Results/Tab Transformer/Test Results/ed16_ep35_predicted.csv, test_acc=0.6085

Running: embed=16, epochs=50
Epoch 1/50 | embed=16: Train Acc=0.8555 | Test Acc=0.5434
Epoch 2/50 | embed=16: Train Acc=0.8654 | Test Acc=0.5570
Epoch 3/50 | embed=16: Train Acc=0.8686 | Test Acc=0.5789
Epoch 4/50 | embed=16: Train Acc=0.8716 | Test Acc=0.6024
Epoch 5/50 | embed=16: Train Acc=0.8729 | Test Acc=0.5966
Epoch 6/50 | embed=16: Train Acc=0.8738 | Test Acc=0.6090
Epoch 7/50 | embed=16: Train Acc=0.8744 | Test Acc=0.6007
Epoch 8/50 | embed=16: Train Acc=0.8768 | Test Acc=0.5935
Epoch 9/50 | embed=16: Train Acc=0.8764 | Test Acc=0.6067
Epoch 10/50 | embed=16: Train Acc=0.8787 | Test Acc=0.5468
Epoch 11/50 | embed=16: Train Acc=0.8792 | Test Acc=0.6142
Epoch 12/50 | embed=16: Train Acc=0.8803 | Te

/tmp/ipython-input-20-1905850978.py:73: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(np.log(probs[np.arange(len(y_true)), y_true] + 1e-12))
/tmp/ipython-input-20-1905850978.py:73: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(np.log(probs[np.arange(len(y_true)), y_true] + 1e-12))


Saved CM at /content/drive/MyDrive/Big Data Analysis and Project - a1906525/Results/Tab Transformer/Confusion Matrices/ed32_ep35.png, preds at /content/drive/MyDrive/Big Data Analysis and Project - a1906525/Results/Tab Transformer/Test Results/ed32_ep35_predicted.csv, test_acc=0.6181

Running: embed=32, epochs=50
Epoch 1/50 | embed=32: Train Acc=0.8575 | Test Acc=0.6152
Epoch 2/50 | embed=32: Train Acc=0.8681 | Test Acc=0.6734
Epoch 3/50 | embed=32: Train Acc=0.8720 | Test Acc=0.6707
Epoch 4/50 | embed=32: Train Acc=0.8745 | Test Acc=0.6706
Epoch 5/50 | embed=32: Train Acc=0.8758 | Test Acc=0.6600
Epoch 6/50 | embed=32: Train Acc=0.8774 | Test Acc=0.6744
Epoch 7/50 | embed=32: Train Acc=0.8775 | Test Acc=0.6817
Epoch 8/50 | embed=32: Train Acc=0.8793 | Test Acc=0.6842
Epoch 9/50 | embed=32: Train Acc=0.8812 | Test Acc=0.6574
Epoch 10/50 | embed=32: Train Acc=0.8830 | Test Acc=0.6381
Epoch 11/50 | embed=32: Train Acc=0.8843 | Test Acc=0.6392
Epoch 12/50 | embed=32: Train Acc=0.8863 | Te

...

Saved CM at /content/drive/MyDrive/Big Data Analysis and Project - a1906525/Results/Tab Transformer/Confusion Matrices/ed128_ep20.png, preds at /content/drive/MyDrive/Big Data Analysis and Project - a1906525/Results/Tab Transformer/Test Results/ed128_ep20_predicted.csv, test_acc=0.5974

Running: embed=128, epochs=35
Epoch 1/35 | embed=128: Train Acc=0.8661 | Test Acc=0.7069
Epoch 2/35 | embed=128: Train Acc=0.8711 | Test Acc=0.6924
Epoch 3/35 | embed=128: Train Acc=0.8740 | Test Acc=0.6524
Epoch 4/35 | embed=128: Train Acc=0.8749 | Test Acc=0.6556
Epoch 5/35 | embed=128: Train Acc=0.8769 | Test Acc=0.6458
Epoch 6/35 | embed=128: Train Acc=0.8794 | Test Acc=0.6581
Epoch 7/35 | embed=128: Train Acc=0.8818 | Test Acc=0.6336
Epoch 8/35 | embed=128: Train Acc=0.8834 | Test Acc=0.6325
Epoch 9/35 | embed=128: Train Acc=0.8844 | Test Acc=0.5890
Epoch 10/35 | embed=128: Train Acc=0.8870 | Test Acc=0.6104
Epoch 11/35 | embed=128: Train Acc=0.8877 | Test Acc=0.6120
Epoch 12/35 | embed=128: Train 

/tmp/ipython-input-20-1905850978.py:73: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(np.log(probs[np.arange(len(y_true)), y_true] + 1e-12))
/tmp/ipython-input-20-1905850978.py:73: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(np.log(probs[np.arange(len(y_true)), y_true] + 1e-12))


Saved CM at /content/drive/MyDrive/Big Data Analysis and Project - a1906525/Results/Tab Transformer/Confusion Matrices/ed128_ep35.png, preds at /content/drive/MyDrive/Big Data Analysis and Project - a1906525/Results/Tab Transformer/Test Results/ed128_ep35_predicted.csv, test_acc=0.6172

Running: embed=128, epochs=50
Epoch 1/50 | embed=128: Train Acc=0.8628 | Test Acc=0.6467
Epoch 2/50 | embed=128: Train Acc=0.8710 | Test Acc=0.6711
Epoch 3/50 | embed=128: Train Acc=0.8732 | Test Acc=0.6705
Epoch 4/50 | embed=128: Train Acc=0.8747 | Test Acc=0.6511
Epoch 5/50 | embed=128: Train Acc=0.8773 | Test Acc=0.6485
Epoch 6/50 | embed=128: Train Acc=0.8791 | Test Acc=0.6877
Epoch 7/50 | embed=128: Train Acc=0.8818 | Test Acc=0.5983
Epoch 8/50 | embed=128: Train Acc=0.8841 | Test Acc=0.6334
Epoch 9/50 | embed=128: Train Acc=0.8848 | Test Acc=0.6084
Epoch 10/50 | embed=128: Train Acc=0.8874 | Test Acc=0.5503
Epoch 11/50 | embed=128: Train Acc=0.8886 | Test Acc=0.5970
Epoch 12/50 | embed=128: Train 

/tmp/ipython-input-20-1905850978.py:73: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(np.log(probs[np.arange(len(y_true)), y_true] + 1e-12))
/tmp/ipython-input-20-1905850978.py:73: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(np.log(probs[np.arange(len(y_true)), y_true] + 1e-12))


Saved CM at /content/drive/MyDrive/Big Data Analysis and Project - a1906525/Results/Tab Transformer/Confusion Matrices/ed128_ep50.png, preds at /content/drive/MyDrive/Big Data Analysis and Project - a1906525/Results/Tab Transformer/Test Results/ed128_ep50_predicted.csv, test_acc=0.5924

All hyperparameter runs complete. Summary at /content/drive/MyDrive/Big Data Analysis and Project - a1906525/Results/Tab Transformer/Performance Metrics/TabTransformer_all_results.xlsx
